This program gives an output of all monsters with a Challenge Rating between x and y (which are user inputs).

In [127]:
import requests
import pprint
import pandas as pd

response = requests.get("http://dnd5eapi.co/api/monsters/")
data = response.json()
results = data["results"]
#This block converts the API into the variable results, which spits out monsters data in JSON format
                               
cr_list = [] #pre-allocate list of all challenge ratings
names_list = [] # pre-allocate list
count = 30 #data['count'] # gets the whole monster list length

#This whole for loop 
for i in range(0, count):
    names_i = results[i]['name']
    names_list.append(names_i)
    url_i = results[i]['url']
    monster_data_raw = requests.get(url_i)
    monster_data = monster_data_raw.json()
    cr_list.append(monster_data["challenge_rating"]) 
#now names_list and cr_list are filled with monster names and challenge ratings
    
# print(list(zip(names_list, cr_list)))
dd_df = pd.DataFrame.from_records(zip(names_list, cr_list))
#This uses pandas to put names and crs into a dataframe with rows and columns
dd_df.columns =["Name", "CR"]
dd_df.index += 1 
#Renames the columns from their default of 0 and 1
dd_df.head(n=3)
#This just returns the first 3 rows to make sure everything looks good


,Name,CR
1,Aboleth,10.00
2,Acolyte,0.25
3,Adult Black Dragon,14.00


In [128]:
def get_inputs():
#This function gets the minimum and maxium from the user and returns the resulting range
    min_cr = int(input("Type a minimum between 0 and 30: "))
    max_cr = int(input("Type a maximum between 0 and 30: "))
    return (min_cr, max_cr)

range_cr = get_inputs()
min_cr = range_cr[0]
max_cr = range_cr[1]
#This runs get inputs (getting the range) then re-defines min_cr and max_cr outside of the function
#Now I can use these inputs in the dataframe for a tidy panda output

print("Here are all monsters with a CR between "+str(min_cr)+" and "+str(max_cr)+":")
dd_df[(dd_df["CR"]>min_cr)&(dd_df["CR"]<=max_cr)]
#dd_df is that dataframe based on cr_list and names_list - we're pulling it up within the range

Type a minimum between 0 and 30: 0
Type a maximum between 0 and 30: 4
Here are all monsters with a CR between 0 and 4:


,Name,CR
2,Acolyte,0.25
26,Animated Armor,1.00
27,Ankheg,2.00
28,Ape,0.50


In [129]:
#Optional: print the stats of the monster at a specified index

foo = input("Type the index of the monster you want stats for: ")
specify_monster = requests.get("http://dnd5eapi.co/api/monsters/"+str(foo))
monster_at_index = specify_monster.json()
pprint.pprint(monster_at_index)

Type the index of the monster you want stats for: 26
{'_id': '5bce91465b7768e7920181ba',
 'actions': [{'attack_bonus': 0,
              'desc': 'The armor makes two melee attacks.',
              'name': 'Multiattack'},
             {'attack_bonus': 4,
              'damage_bonus': 2,
              'damage_dice': '1d6',
              'desc': 'Melee Weapon Attack: +4 to hit, reach 5 ft., one '
                      'target. Hit: 5 (1d6 + 2) bludgeoning damage.',
              'name': 'Slam'}],
 'alignment': 'unaligned',
 'armor_class': 18,
 'challenge_rating': 1,
 'charisma': 1,
 'condition_immunities': 'blinded, charmed, deafened, exhaustion, frightened, '
                         'paralyzed, petrified, poisoned',
 'constitution': 13,
 'damage_immunities': 'poison, psychic',
 'damage_resistances': '',
 'damage_vulnerabilities': '',
 'dexterity': 11,
 'hit_dice': '6d8',
 'hit_points': 33,
 'index': 26,
 'intelligence': 1,
 'languages': '',
 'name': 'Animated Armor',
 'senses': 'blindsig

In [130]:
# Optional: This gets the name of a random monster
random_monster = dd_df.sample(n=1, random_state=1)
#random state of 1 preserves the output, random state 0 will change the output every run
print(random_monster)

monster_index = random_monster.index.values.astype(int)
#This stores the index of that random monster

                     Name    CR
18  Ancient Bronze Dragon  22.0


In [131]:
#If you then want all of that random monster's stats, printed nicely with the "pretty print" module:
response_random = requests.get("http://dnd5eapi.co/api/monsters/"+str(monster_index[0]))
data_random = response_random.json()
pprint.pprint(data_random)

{'_id': '5bce91465b7768e7920181b4',
 'actions': [{'attack_bonus': 0,
              'desc': 'The dragon can use its Frightful Presence. It then '
                      'makes three attacks: one with its bite and two with its '
                      'claws.',
              'name': 'Multiattack'},
             {'attack_bonus': 16,
              'damage_bonus': 9,
              'damage_dice': '2d10',
              'desc': 'Melee Weapon Attack: +16 to hit, reach 15 ft., one '
                      'target. Hit: 20 (2d10 + 9) piercing damage.',
              'name': 'Bite'},
             {'attack_bonus': 16,
              'damage_bonus': 9,
              'damage_dice': '1d6',
              'desc': 'Melee Weapon Attack: +16 to hit, reach 10 ft., one '
                      'target. Hit: 16 (2d6 + 9) slashing damage.',
              'name': 'Claw'},
             {'attack_bonus': 0,
              'damage_bonus': 9,
              'damage_dice': '2d8',
              'desc': 'Melee Weapon Attack: 